<a href="https://colab.research.google.com/github/gbarra2511/SLM-project/blob/main/SLMproject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1: Starting to build de small language model
This project is based in Dr Raj Dandekar(MIT PhD in Machine learning) teachings. Our goal is to build a production level SLM, but for instance I'm are developing and documenting the process of building an slm from scratch.

STEPS:

1.   Creating Dataset
2.   Tokenizing the dataset
3.   Creating input target pairs
4.   Creating the SLM archtecture
5.   Setting up for pre training
6.   pre training
7.   Inferences

there are many processes included in these steps, this is only a resume.

In [1]:
!pip install datasets

In [2]:
!pip install -U datasets fsspec huggingface_hub


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.3/515.3 kB 42.7 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: huggingface_hub
    Found existing installation: huggingface-hub 0.33.2
    Uninstalling huggingface-hub-0.33.2:
      Successfully uninstalled huggingface-hub-0.33.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_s

In [3]:
from datasets import load_dataset

ds = load_dataset("roneneldan/TinyStories")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

(…)-00000-of-00004-2d5a1467fff1081b.parquet:   0%|          | 0.00/249M [00:00<?, ?B/s]

(…)-00001-of-00004-5852b56a2bd28fd9.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00002-of-00004-a26307300439e943.parquet:   0%|          | 0.00/246M [00:00<?, ?B/s]

(…)-00003-of-00004-d243063613e5a057.parquet:   0%|          | 0.00/248M [00:00<?, ?B/s]

(…)-00000-of-00001-869c898b519ad725.parquet:   0%|          | 0.00/9.99M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2119719 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/21990 [00:00<?, ? examples/s]

#tokenizing the dataset using BPE (byte pair encoding)

Goal: tokenize the dataset and store it's ID's in a single .bin file


*   Tokenize in tokenIDs
*   Create a fille called "traind.bin" and "validation.bin" where it will be stored the tokenIDs
*   make sure that the tokenIDs are stored on a disk, rather than on the RAM for efficient computation



In [4]:
!pip install tokenizers
import tiktoken
import os
import numpy as np
from tqdm.auto import tqdm

enc = tiktoken.get_encoding("gpt2")

# take a look at https://github.com/karpathy/nanoGPT/blob/data/openwebtext/prepare.py

def process(example):
    ids = enc.encode_ordinary(example['text']) #encode_originary ignores special tokens
    out = {'ids': ids, 'len': len(ids)}
    return out

if not os.path.exists("train.bin"):
    tokenized = ds.map(process, remove_columns=['text'], desc="tokenizing the splits", num_proc=8)
    for split, dset in tokenized.items():
      arr_len = np.sum(dset['len'], dtype=np.uint64)
      filename = f'{split}.bin'
      dtype = np.uint16 #can do since enc.max_token_value == 50256 is < 2**16
      arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
      total_batches = 1024

      idx = 0
      for batch_idx in tqdm(range(total_batches), desc =f'writing{filename}'):
        #batch together samples for faster write
        batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
        arr_batch = np.concatenate(batch['ids'])
        #Write to mmap
        arr[idx : idx + len(arr_batch)] = arr_batch
        idx += len(arr_batch)
      arr.flush()



tokenizing the splits (num_proc=8):   0%|          | 0/2119719 [00:00<?, ? examples/s]

tokenizing the splits (num_proc=8):   0%|          | 0/21990 [00:00<?, ? examples/s]

writingtrain.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

writingvalidation.bin:   0%|          | 0/1024 [00:00<?, ?it/s]

# **STEP 3** creating our input- output pairs

For doing that we need to determine our context_size( lenght of words that my language model is looking at one time before predicting the next token) -- divide the text into chunks of words and the maximum number of words in the chunks is the context_size

we need to determine thought our batch_size( we dont process all data at once, so we first take the output of the first batch, calculate the loss, propagate it backwards and updare the parameters, than do it again for all batches)

#note: the output is going to be the input shifted to the right:
example: one day a little girl named lily(phrase)
input: one day a little girl named
ouput: day a little girl named lily


its trying to predict the next word (Lily), the context_size in this case is 6.The target is Lily, but the output can be different and thats why we calculate the loss, to try to minimize it.

In [5]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import math
from dataclasses import dataclass
import numpy as np
from contextlib import nullcontext
import os

 #non_blocking=TRUE allows the cpu to do other stuff while the process is going on


# Some functions from https://github.com/karpathy/nanoGPT/blob/master/train.py with slight modifications
#whenever get_batch is defined, we can create input  baches and output batches by moving right by 1, every get_batch function return an input and an output matriz.
def get_batch(split):
    # We recreate np.memmap every batch to avoid a memory leak, as per
    # https://stackoverflow.com/questions/45132940/numpy-memmap-memory-usage-want-to-iterate-once/61472122#61472122
    if split == 'train':
        data = np.memmap('train.bin', dtype=np.uint16, mode='r')
    else:
        data = np.memmap('validation.bin', dtype=np.uint16, mode='r')
    #here we take four random int that correspond to four different positions because batch_size == 4
    #block_size is the context_size that is mentioned before
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #x and y are stacking together the vectors x1, x2, x3, x4 , y1,...y4
    x = torch.stack([torch.from_numpy((data[i:i+block_size]).astype(np.int64)) for i in ix])
    #see that y is only x shifted to ridght by 1,
    y = torch.stack([torch.from_numpy((data[i+1:i+1+block_size]).astype(np.int64)) for i in ix])


    #NOTE: CUDA is a platform and software created by nvidia that allows GPU's to process general purpose stuff and not only graphics
    if device_type == 'cuda':
        # pin arrays x,y locks the memory of the tensor in RAM and this allows faster transfer of tensor to GPU, because it reserves some memory in RAM.
        #non_blocking=TRUE allows the cpu to do other stuff while the process is going on
        x, y = x.pin_memory().to(device, non_blocking=True), y.pin_memory().to(device, non_blocking=True)
    else:
        x, y = x.to(device), y.to(device)
    return x, y



# **step 4** - defing the SLM Model Architecture

there are multiple different things happenning:

longest code cell
most important part


1.   input block
2.   processor block
3.   output block

#At the end we are feeding a sequence of words and we are trying to predict the next word or token

# **Input Block**:
**token embedding**: we convert every TokenID's into an multidimensional vector, words have sort of an semmantic notion which is very important to the language model to capture. Vectors next to each other will be similar in meaning. We maintaing a token embedding matriz that the column size is the vocabulary_size, example, english is 50000 words, and each TokenID is encoded as an high dimensional vector, the dimension of the token(embedding dimension) is choosen by me, and its maintenned throught the Slm architechture. The token embedding matrix serves as a lookup table, so when we have TokenID's we can look at the matrix and retrieve the corresponding vector.

#**Transformer block**
**layer normalization**: first step to prevent different distribution by back propagation, so its stablishes a range(internal coehrent shift) to improve training performance. PyTorch( layer_norm ), normalizes rows

**Multi-head attention**:
in the following phrases "The dog chased the ball. It could not catch it."

the first it is reffering to the dog, the second one is reffering to the ball, but how the model can differentiate, and stabilishes relations between words?

Attention mechanism **augments the inputs embedding vector** so for every vector it can have information of the neighboors. We capture, basically, the attention scores between the token and all the tokens arround it. We want to transform the normal vector into a context vector, because it is much richer in information than the other, it has the information about how the word relates to all the other words arround it.

quick and superficial recap of the process, we have the matrix called query, key and values. The called atttention weight matrix is given by doing dot product(escalar) between query and key, the coordinate stabilishes the relation between the Xn and Yn values, where Xn and Yn are words, for example: point A12 = 0,7 where x1 is One and Y2 is day, so 0,7 is the atttention weight between this words. Multiplying this matrix with the value matrix is given the context matrix.

**Dropout**
neurons ar turned-off randomly during training to **not overfit the model** and guarantee that the model is really learning. How it occurs? the output of some neurons are turned to zero

**Feed Forward Neural Network**
without this the model cannot leran the patterns in the underlying data. Every time in deep learning if a model is not working in a small dimensional space, we project it to an high dimensional space so it can capture and understand an non-linearities. So we exppand the model in the intermidiate layers than compress for the output, so it can have the same size as the input layer.
The activation function here is GeLU. Hyperparameter, very good results.

**Shortcuts**
The input of the first layer is added to the output of the first layer, and it turns into the input of the next layer.




In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math
from dataclasses import dataclass
import numpy as np
from tqdm.auto import tqdm
from contextlib import nullcontext
import os

class LayerNorm(nn.Module):
    def __init__(self, ndim, bias):
        super().__init__()
        self.weight = nn.Parameter(torch.ones(ndim))
        self.bias = nn.Parameter(torch.zeros(ndim)) if bias else None
    def forward(self, x):
        return F.layer_norm(x, self.weight.shape, self.weight, self.bias, 1e-5)

class CausalSelfAttention(nn.Module):
    def __init__(self, config):
        super().__init__()
        assert config.n_embd % config.n_head == 0
        self.c_attn = nn.Linear(config.n_embd, 3 * config.n_embd, bias=config.bias)
        self.c_proj = nn.Linear(config.n_embd, config.n_embd, bias=config.bias)
        self.attn_dropout = nn.Dropout(config.dropout)
        self.resid_dropout = nn.Dropout(config.dropout)
        self.n_head = config.n_head
        self.n_embd = config.n_embd
        self.flash = hasattr(F, 'scaled_dot_product_attention')
        if not self.flash:
            self.register_buffer("bias", torch.tril(torch.ones(config.block_size, config.block_size))
                                       .view(1, 1, config.block_size, config.block_size))

    def forward(self, x):
        B, T, C = x.size()
        q, k, v = self.c_attn(x).split(self.n_embd, dim=2)
        k = k.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        q = q.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)
        v = v.view(B, T, self.n_head, C // self.n_head).transpose(1, 2)

        if self.flash:
            y = F.scaled_dot_product_attention(q, k, v, attn_mask=None, dropout_p=self.attn_dropout.p if self.training else 0.0, is_causal=True)
        else:
            att = (q @ k.transpose(-2, -1)) * (1.0 / math.sqrt(k.size(-1)))
            att = att.masked_fill(self.bias[:, :, :T, :T] == 0, float('-inf'))
            att = F.softmax(att, dim=-1)
            att = self.attn_dropout(att)
            y = att @ v

        y = y.transpose(1, 2).contiguous().view(B, T, C)
        y = self.resid_dropout(self.c_proj(y))
        return y

class MLP(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.c_fc = nn.Linear(config.n_embd, 4 * config.n_embd, bias=config.bias)
        self.gelu = nn.GELU()
        self.c_proj = nn.Linear(4 * config.n_embd, config.n_embd, bias=config.bias)
        self.dropout = nn.Dropout(config.dropout)
    def forward(self, x):
        return self.dropout(self.c_proj(self.gelu(self.c_fc(x))))

class Block(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.ln1 = LayerNorm(config.n_embd, config.bias)
        self.attn = CausalSelfAttention(config)
        self.ln2 = LayerNorm(config.n_embd, config.bias)
        self.mlp = MLP(config)
    def forward(self, x):
        x = x + self.attn(self.ln1(x))
        x = x + self.mlp(self.ln2(x))
        return x

@dataclass
class GPTConfig:
    block_size: int
    vocab_size: int
    n_layer: int
    n_head: int
    n_embd: int
    dropout: float = 0.0
    bias: bool = True
#class gpt is the complete model where its taking the blocks and the embedding and
class GPT(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.config = config
        self.transformer = nn.ModuleDict(dict(
            #wte = word token embedding-> convert words tokens into vectors
            wte=nn.Embedding(config.vocab_size, config.n_embd),
            #wpe = word position embedding-> creates a vector for every word position, putting like an order
            wpe=nn.Embedding(config.block_size, config.n_embd),
            drop=nn.Dropout(config.dropout),
            h=nn.ModuleList([Block(config) for _ in range(config.n_layer)]),
            ln_f=LayerNorm(config.n_embd, config.bias),
        ))
        #projects the transformer output in the size of vocabulary creating the logits
        self.lm_head = nn.Linear(config.n_embd, config.vocab_size, bias=False)
        # weight tying-> the embedding layer and the output layer has the same weights(better efficiency and reduces parameters)
        self.transformer.wte.weight = self.lm_head.weight

        self.apply(self._init_weights)
        for pn, p in self.named_parameters():
            if pn.endswith('c_proj.weight'):
                nn.init.normal_(p, mean=0.0, std=0.02 / math.sqrt(2 * config.n_layer))

    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            nn.init.normal_(module.weight, mean=0.0, std=0.02)

    #here its being defined the foward pass
    def forward(self, idx, targets=None):
        #the token id(idx) are converted into vectors by wte and wpe, they are being added together.

        device = idx.device
        b, t = idx.size()
        assert t <= self.config.block_size
        pos = torch.arange(0, t, dtype=torch.long, device=device)

        tok_emb = self.transformer.wte(idx)
        pos_emb = self.transformer.wpe(pos)

        x = self.transformer.drop(tok_emb + pos_emb)
        #here the result x is passing in every block and than is normalized
        for block in self.transformer.h:
            x = block(x)
        x = self.transformer.ln_f(x)
        #if targets are being
        if targets is not None:
            logits = self.lm_head(x)
            loss = F.cross_entropy(logits.view(-1, logits.size(-1)), targets.view(-1), ignore_index=-1)
            return logits, loss
        else:
            logits = self.lm_head(x[:, [-1], :])
            return logits, None

    @torch.no_grad()
    def generate(self, idx, max_new_tokens, temperature=1.0, top_k=None):
        """
        Generate tokens given a conditioning sequence.
        idx: Tensor of shape (B, T)
        """
        for _ in range(max_new_tokens):
            idx_cond = idx if idx.size(1) <= self.config.block_size else idx[:, -self.config.block_size:]
            logits, _ = self(idx_cond)
            logits = logits[:, -1, :] / temperature
            if top_k is not None:
                v, _ = torch.topk(logits, min(top_k, logits.size(-1)))
                logits[logits < v[:, [-1]]] = -float('Inf')
            probs = F.softmax(logits, dim=-1)
            idx_next = torch.multinomial(probs, num_samples=1)
            idx = torch.cat((idx, idx_next), dim=1)
        return idx

#OUTPUT

there is other normilazation. The output head it is another neural network that convert every vector into vocab_size. After passing the output N.N it is formed the logits matrix or tensor, where every element corresponds to the probability of that line(id) being the next token.

**How it is used?And why?**

in the batch logits tensor it is applied softmax function to give the probabilities, them we take the highest value of each line(highest probability- output), where each line is an word, then calculates the loss betwween this output and the given target. This loss is used to correct the weights in backpropagation. In the training routine we need to minimize this loss function using cross entropy.

In [7]:
config = GPTConfig(
    vocab_size=50257,     # use the tokenizer's vocab size
    block_size=128,       # or whatever context size you're training with
    n_layer=6,
    n_head=6,
    n_embd=384,
    dropout=0.1,
    bias=True
)

model = GPT(config)

#defining the loss function


In [8]:
def estimate_loss(model):
    out = {}
    model.eval()
    with torch.inference_mode():
        for split in ['train', 'val']:
            losses = torch.zeros(eval_iters)
            for k in range(eval_iters):
                X, Y = get_batch(split)
                with ctx:
                    logits, loss = model(X, Y)
                losses[k] = loss.item()
            out[split] = losses.mean()
    model.train()
    return out

In [9]:
# Training Config
import torch
from contextlib import nullcontext

learning_rate = 1e-4 #more stable training, earlier 1e-4
max_iters = 20000 #increase from 25000
warmup_steps = 1000 #smoother initial train, earlier 100
min_lr = 5e-4 #lower rate, earlier 5e-4
eval_iters = 500 # increased from 100
batch_size = 32 # changed from 16, better gradient estimate
block_size = 128 #changed from 64, capture longer range dependencies

gradient_accumulation_steps = 32 # reduced from 50

device =  "cuda" if torch.cuda.is_available() else "cpu"
device_type = 'cuda' if 'cuda' in device else 'cpu' # for later use in torch.autocast
# note: float16 data type will automatically use a GradScaler

#dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32', 'bfloat16', or 'float16', the latter will auto implement a GradScaler
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
#amp is automatic mixed precision, it enables to use float16 when its safe and float32 when its needed more precision(MATRIX MULTIPLICATIONS AND DROPOUT GELU)
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

torch.set_default_device(device)
torch.manual_seed(42)

In [10]:
from torch.optim.lr_scheduler import LinearLR,SequentialLR, CosineAnnealingLR

##PUT IN WEIGHT DECAY, CHANGED BETA2 to 0.95
#THE ERROR IS BEING AJUSTED(FINAL STEP OF BACKPROPAGATION)
optimizer =  torch.optim.AdamW(model.parameters(), lr=learning_rate, betas=(0.9, 0.95), weight_decay=0.1, eps=1e-9) #weight decay for regularization

scheduler_warmup = LinearLR(optimizer, total_iters = warmup_steps) #Implement linear warmup
scheduler_decay = CosineAnnealingLR(optimizer,T_max = max_iters - warmup_steps, eta_min = min_lr) #Implement lr decay
scheduler = SequentialLR(optimizer, schedulers=[scheduler_warmup, scheduler_decay], milestones=[warmup_steps]) #Switching from warmup to decay

# https://stackoverflow.com/questions/72534859/is-gradscaler-necessary-with-mixed-precision-training-with-pytorch
scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))

/tmp/ipython-input-10-2132813893.py:11: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=(dtype == 'float16'))


#Training loop

**explaining back propagation**:
is the process which neural networks learns from their mistakes. Allows you to know exactly the steepness of the gradient for every setting, so you know precisely how to adjust the weights for a better perfomance.

There are four main steps:

1.   Forward pass
2.   Calculating error
3.   Backward Pass
4.   Adjusting error

At the final layer of the network it takes an guess, calculate the error using some function, in this case softmax, then move backwards layer by layer calculating with the chain rule(CALCULUS) how much each knob contributed for the result. If a knob is steep, it will have a big impact on the final result, it is more important. If a knob is shallow, it has less importance for the final result.

When the backward pass gets in the first layer, there is a gradient assigned to every knob(adjustable parameter). The next step is adjust for a better performance.

Updating weights: changes the weight in the opposite direction(gradient descent) using the learning rate as parameter. For example: if knobX weight increases, the learning rate decreases, so after the process it will adjust knobX weight lower so the learning rate can get higher.

After thousands or millions of times the network gets better.



In [1]:
best_val_loss = float('inf')
best_model_params_path = "best_model_params.pt"
train_loss_list, validation_loss_list = [], []

# Ensure model is on the correct device
model = model.to(device)

# In your training loop
for epoch in tqdm(range(max_iters)):
    if epoch % eval_iters == 0 and epoch != 0:
        # Ensure estimate_loss uses the correct device
        losses = estimate_loss(model)
        print(f"Epoch {epoch}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")
        print(f"The current learning rate: {optimizer.param_groups[0]['lr']:.5f}")
        train_loss_list += [losses['train']]
        validation_loss_list += [losses['val']]

        if losses['val'] < best_val_loss:
            best_val_loss = losses['val']
            torch.save(model.state_dict(), best_model_params_path)

    # Ensure X and y are on the correct device
    X, y = get_batch("train")
    X, y = X.to(device), y.to(device)

    with ctx:
        logits, loss = model(X, y)
        loss = loss / gradient_accumulation_steps
        scaler.scale(loss).backward()

    if ((epoch + 1) % gradient_accumulation_steps == 0) or (epoch + 1 == max_iters):
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
        scaler.step(optimizer)
        scaler.update()
        optimizer.zero_grad(set_to_none=True)
    scheduler.step()

NameError: name 'model' is not defined

In [ ]:
import matplotlib as plt
train_loss_list_converted = [i.cpu().detach() for i in train_loss_list]
validation_loss_list_converted = [i.cpu().detach() for i in validation_loss_list]

plt.plot(train_loss_list_converted, 'g', label = 'train_loss')
plt.plot(validation_loss_list_converted, 'r', label = 'validation_loss')
plt.xlabel("Steps- Every 100 epochs")
plt.ylabel("loss")
plt.legend()
plt.show()


In [ ]:
#loading the model
model = GPT(config)
device = "cuda" if torch.cuda.is_available() else "cpu"
best_model_params_path = "best_model_params.pt"
model.load_state_dict(torch.load(best_model_params_path, map_location=torch.device(device)))


In [ ]:
sentence = "Once upon a time there was a pumpkin"
context = (torch.tensor(enc.encode_ordinary(sentence).unsqueeze(dim = 0)))
y = model.generate(context, 200)
print(enc.decode(y.squeeze().tolist()))

In [ ]:
sentence = "A little girl went to the woods"
context = (torch.tensor(enc.encode_ordinary(sentence).unsqueeze(dim = 0)))
y = model.generate(context, 200)
print(enc.decode(y.squeeze().tolist()))

In [ ]:
from google.colab import runtime
runtime.unassign